<b>COSMIC RADIATION: XGBoost</b>
    
rodrigoX: Contains 4 parameters
    1. NALLParticlesTotal : Total number of particles generated by the event in the ground level.
    2. MUTotal : Total number of muons.
    3. ELTotal : Total number of electromagnetic particles.
    4. Zenith : Zenith angle of the particle [degrees].
    5. Energy : Particle energy [GeV].

rodrigoY: contains the target
    Labels: photon, proton, helium, nitrogen, iron.
        the smallest number is assigned to photon (A=0) and the highest number to the heaviest particle, this is, iron (B=4).
        
        0==photon     1==proton    2==helium     3==nitrogen   4==iron 
     
     

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

### data


In [2]:
#leave the dataset ready
df_x = pd.read_fwf('XRodrigo.txt')
df_x.columns = ["NALLParticlesTotal", "MUTotal", "ELTotal", "Zenith","Energy"]
df_y=pd.read_fwf('YRodrigo.txt')
df_y.columns = ["Particle"]

transpose=df_x.T
transpose_y=df_y.T #Transpose to put them together
df_tot = transpose.append(transpose_y)
df_tot=df_tot.T.sample(frac=1).reset_index(drop=True)#este el el df completo y barajado

df_y=df_tot[['Particle']]
df_x=df_tot[["NALLParticlesTotal", "MUTotal", "ELTotal", "Zenith","Energy"]]

In [3]:
#train-test split and reescale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, random_state=0,test_size = 0.25)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_test_np=y_test.to_numpy()
y_train_np=y_train.to_numpy().flatten()


### grid search

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 2, 5],
        'subsample': [0.6, 1.0],
        'colsample_bytree': [0.6, 1.0],
        'max_depth': [3, 7,19]
        }

grid = GridSearchCV(XGBClassifier(), params,  n_jobs=-1,cv=5, verbose=1 )

grid_result=grid.fit(X_train, y_train)

In [12]:
print(grid_result)
print("The best parameters are %s with a score of %0.5f"
      % (grid_result.best_params_, grid_result.best_score_))
results = pd.DataFrame(grid.cv_results_)
results.to_csv('grid-search-results-01.csv', index=False)


 All results:
{'mean_fit_time': array([200.70774627, 171.60928807, 128.31872134, 125.5676528 ,
       147.13994484]), 'std_fit_time': array([ 1.34377288, 31.33442208,  3.79062406,  0.3041845 , 32.59047517]), 'mean_score_time': array([13.38582678,  8.63449774,  8.34216428,  8.38539371,  6.64857764]), 'std_score_time': array([2.1475013 , 0.30771935, 0.12807829, 0.12464314, 2.79805531]), 'param_subsample': masked_array(data=[1.0, 1.0, 1.0, 1.0, 1.0],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_min_child_weight': masked_array(data=[1, 10, 10, 5, 10],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_depth': masked_array(data=[3, 3, 3, 3, 3],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_gamma': masked_array(data=[1, 1, 2, 0.5, 0.5],
             mask=[False, False, False, False, False],
     

### model

In [4]:
# fit model on training data
model = XGBClassifier(gamma=0.1,learning_rate=0.01, n_estimators=1000, objective='multi:softmax',verbosity=1, max_depth= 19)
model=model.fit(X_train, y_train)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\HP\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [7]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[2186    2    0    0    0]
 [   0 1937  134   14    0]
 [   0   48 1907  115    2]
 [   0   13  137 1937   37]
 [   0   10   11   54 1954]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2188
         1.0       0.96      0.93      0.95      2085
         2.0       0.87      0.92      0.90      2072
         3.0       0.91      0.91      0.91      2124
         4.0       0.98      0.96      0.97      2029

    accuracy                           0.95     10498
   macro avg       0.95      0.94      0.95     10498
weighted avg       0.95      0.95      0.95     10498

0.9450371499333207
